<a href="https://colab.research.google.com/github/umSamWeiHong/DepressionModel/blob/master/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install gradio

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.special import softmax
from tensorflow.keras.models import load_model

import joblib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

model = load_model('/content/drive/MyDrive/Machine Learning Group Project/neural_network_model.h5')
scaler = joblib.load('/content/drive/MyDrive/Machine Learning Group Project/neural_network_scaler.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import gradio as gr

In [5]:
def fn(marital, marital_problems, religion_help, routine_loss, stress_class_disrupted,
       medical_illness, mental_illness, covid_symptoms, residence_high_covid, quarantine,
       expenses, living_with, _,
       q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12):
    
    family = (q3 + q4 + q8 + q11)
    friends = (q6 + q7 + q9 + q12)
    significant_others = (q1 + q2 + q5 + q10)
    
    X = [marital, marital_problems, religion_help, routine_loss, stress_class_disrupted,
         medical_illness, mental_illness, covid_symptoms, residence_high_covid, quarantine,
         family, friends, significant_others]

    if (living_with == 0): 
      X += [1, 0, 0]
    elif (living_with == 1):
       X += [0, 1, 0]
    elif (living_with == 2):
       X += [0, 0, 1]

    if (expenses == 0): 
      X += [1, 0, 0]
    elif (expenses == 1):
       X += [0, 1, 0]
    elif (expenses == 2):
       X += [0, 0, 1]

    X = pd.DataFrame(X).T
    X.columns = ['Marital', 'Marital_problems', 'Religion_help', 'Routine_loss',
       'Stress_class_disrupted', 'Medical_illness', 'Mental_illness',
       'COVID_symptoms', 'Residence_high_COVID', 'Quarantine', 'MSPSS_family',
       'MSPSS_friends', 'MSPSS_significant_others', 'Alone',
       'Friends/Coursemates', 'Family', '< RM1000', 'RM1000 - 3000',
       '> RM3000']
    X = scaler.transform(X)

    y = model.predict(X)
    
    y_classes = ["No depression", "Mild depression", "Moderate depression", "Severe depression", "Very severe depression"]
    y_prob = softmax(y).tolist()[0]
    print(y_prob)
    dictionary = {k: v for k, v in zip(y_classes, y_prob)}
    print(dictionary)

    y = np.argmax(y, axis=1)
    y = y[0]
    output = ""
    if (y == 0):
      output = "No depression"
    if (y == 1):
      output = "Mild depression"
    if (y == 2):
      output = "Moderate depression"
    if (y == 3):
      output = "Severe depression"
    if (y == 4):
      output = "Very severe depression"

    return output, dictionary

app = gr.Interface(fn=fn,
                   inputs=[gr.Radio(label = "Marital status", choices=['Married', 'Single'], type = 'index'),
                           gr.Radio(label = "Do you have marital problem during the lockdown?", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Does religion help you cope during the lockdown?", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Are you frustrated due to loss of daily routine?", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Do you feel stress due to study being disrupted?", choices=['No', 'Yes'], type = 'index'),
                           
                           gr.Radio(label = "History of pre-existing medical illnesses", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "History of pre-existing psychiatric illnesses", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Are you afraid that you might be infected if you develop symptoms?", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Are cases prevalent in your residence area?", choices=['No', 'Yes'], type = 'index'),
                           gr.Radio(label = "Were you quarantined due to exposure to positive cases?", choices=['No', 'Yes'], type = 'index'),

                           gr.Radio(label = "Average monthly expenses", choices=['< RM 1000', 'RM 1000 - RM 3000', '> RM 3000'], type = 'index'),
                           gr.Radio(label = "Who do you live with during the lockdown?", choices=['Alone', 'Friends or coursemates', 'Family'], type = 'index'),

                           gr.Markdown(value = """#### Indicate how you feel about each statement.
                                                  1. Very Strongly Disagree
                                                  2. Strongly Disagree
                                                  3. Mildly Disagree
                                                  4. Neutral
                                                  5. Mildly Agree
                                                  6. Strongly Agree
                                                  7. Very Strongly Agree"""),

                           gr.Slider(label = "There is a special person who is around when I am in need.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "There is a special person with whom I can share joys and sorrows.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "My family really tries to help me.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I get the emotional help & support I need from my family.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I have a special person who is a real source of comfort to me.", minimum = 1, maximum = 7, step = 1),

                           gr.Slider(label = "My friends really try to help me.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I can count on my friends when things go wrong.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I can talk about my problems with my family.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I have friends with whom I can share my joys and sorrows.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "There is a special person in my life who cares about my feelings.", minimum = 1, maximum = 7, step = 1),

                           gr.Slider(label = "My family is willing to help me make decisions.", minimum = 1, maximum = 7, step = 1),
                           gr.Slider(label = "I can talk about my problems with my friends.", minimum = 1, maximum = 7, step = 1),
                   ],
                   outputs=[gr.Textbox(label = "Depression Severity"),
                            gr.Label(label = "Classification Probabilities")])

In [ ]:
app.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://52678.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


[0.14971107244491577, 0.14916963875293732, 0.40302276611328125, 0.14904755353927612, 0.14904898405075073]
{'No depression': 0.14971107244491577, 'Mild depression': 0.14916963875293732, 'Moderate depression': 0.40302276611328125, 'Severe depression': 0.14904755353927612, 'Very severe depression': 0.14904898405075073}
[0.17956258356571198, 0.2568172812461853, 0.2429119199514389, 0.1603381186723709, 0.1603701114654541]
{'No depression': 0.17956258356571198, 'Mild depression': 0.2568172812461853, 'Moderate depression': 0.2429119199514389, 'Severe depression': 0.1603381186723709, 'Very severe depression': 0.1603701114654541}


In [ ]:
app.close()